### Text Selection Data Merge

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Result/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Text Data Read

In [9]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
44995,hayırbu,871
44996,konferansına,871
44997,kardeşisin,871
44998,kaldırıyorsun,871


In [10]:
#df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected.csv")
#df_twogram_select = df_twogram_select.head(1000)
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
213666,olmadan gerçek,52
213667,epey kolay,52
213668,hafızan iyi,52
213669,senin yaşındaydım,52


In [11]:
#df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected.csv")
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
307731,arkadaş mı olmak,38
307732,olan birkaç şey,38
307733,da size anlatayım,38
307734,için olay yerine,38


In [12]:
#df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected.csv")
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
388274,söz veriyorum kurul iş,16
388275,çıkmak için can atıyor,16
388276,ve içinden alevlerin çıktığı,16
388277,ne zaman isterse ama,16


In [13]:
#df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected.csv")
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
122369,ile matrix arasında seçim yapmam,19
122370,hey o iyi bir adam,19
122371,beklemek için nasıl bir yer,19
122372,yani bak normalde beni bilirsin,19


In [14]:
#df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected.csv")
df_sixgram_select

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
300159,bu çocuğun bir süper babaya ihtiyacı,9
300160,sadece yaranın üzerine baskı uygulamaya çalıştım,9
300161,tamam bir iki hafta içinde uğra,9
300162,bana şükran gününde ananas toplama sözü,9


In [15]:
#df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected.csv")
df_sevengram_select

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9
141655,herifi durdurmak için sadece bir saat kırk,9
141656,görücü usulü evlenmek daha çok işe yarıyor,9
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9


In [16]:
#df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected.csv")
df_eightgram_select

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
109469,ve bunu her yaptığında takım arkadaşlarından b...,7
109470,gizli gizli dolaşan genç bir haham imajı canla...,7
109471,için tek yol kahrolası paramı bana geri vermen,7
109472,teşkilatın görev kapsamında olsa bile görev se...,7


In [17]:
#df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected.csv")
df_ninegram_select

,ninegram,frequency
0,dokuz sekiz yedi altı beş dört üç iki bir,463
1,bir iki üç dört beş altı yedi sekiz dokuz,405
2,iki üç dört beş altı yedi sekiz dokuz on,287
3,on dokuz sekiz yedi altı beş dört üç iki,277
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134
...,...,...
40393,artık sana kan nakli yapacak doktor hoffman da...,7
40394,senin ülken için potansiyel terörist teröristi...,7
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7
40396,herkes benim iyi adam olduğuma inanmakta bu ka...,7


In [18]:
#df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected.csv")
df_tengram_select

,tengram,frequency
0,bir iki üç dört beş altı yedi sekiz dokuz on,287
1,on dokuz sekiz yedi altı beş dört üç iki bir,251
2,g i t a r i s y e n,128
3,t o x i c w o r l d,93
4,umutsuzca birinin arkasından koşuyorum daha gü...,76
...,...,...
30518,kendi ülken başkasının ülkesi ya da kendi kişi...,5
30519,son altı ay içinde aynı tarzda bir düzine soyg...,5
30520,benim hikayem birkaç yıl önce yayınlandı kendi...,5
30521,aslında ben bunun pek de kötü bir fikir olmadı...,5


#### Text Data Word-Word Group In Word Group

In [19]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

##### Word

In [20]:
# 2 Way Fast
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "twogram"
df_target = df_twogram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [21]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_two

,word,twogram
0,ettiğim,ettiğim için
1,ettiğim,rahatsız ettiğim
2,ettiğim,ettiğim gibi
3,ettiğim,tahmin ettiğim
4,ettiğim,merak ettiğim
...,...,...
403752,haller,bay haller
403753,haller,haller oldu
403754,boşandıktan,boşandıktan sonra
403755,sabırsızlanıyordum,için sabırsızlanıyordum


In [23]:
df_word_in_two["word"].nunique()

29823

In [24]:
df_word_in_two_list = df_word_in_two.groupby(["word"])["twogram"].apply(list).reset_index(name="twogram")
df_word_in_two_list

,word,twogram
0,a,"[l a, a evet, a j, a b, a aziz, and a, a ve, t..."
1,aa,"[aa evet, şey aa, aa bu, aa şey, aa öyle, aa h..."
2,aaa,[aaa şey]
3,aah,"[aah o, aah en]"
4,aang,"[aang adında, aang adlı]"
...,...,...
29818,şık,"[mı alayım, şık bir, şık görünüyorsun, kadar ş..."
29819,şımarık,"[şımarık bir, şımarık ve, seni şımarık]"
29820,şınav,[şınav çek]
29821,şırınga,[bir şırınga]


In [25]:
#df_word_in_two_list.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

In [26]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [27]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_three = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_three

,word,threegram
0,güvenli,güvenli bir yere
1,güvenli,güvenli bir yer
2,güvenli,güvenli bir şekilde
3,güvenli,güvenli bir yerde
4,güvenli,daha güvenli bir
...,...,...
658708,ayrıntısına,ince ayrıntısına kadar
658709,ayrıntısına,ufak ayrıntısına kadar
658710,ayrıntısına,en ufak ayrıntısına
658711,kardeşisin,kardeşisin değil mi


In [28]:
df_word_in_three_list = df_word_in_three.groupby(["word"])["threegram"].apply(list).reset_index(name="threegram")
df_word_in_three_list

,word,threegram
0,a,"[a ve b, a b c, two and a, a r i, t a r, i t a..."
1,aah,"[aah o günkü, aah en uzun, aah o gün]"
2,aang,"[aang adında bir, aang adlı bir]"
3,ab,[kan grubu ab]
4,abartma,"[kadar da abartma, bu kadar abartma, o kadar a..."
...,...,...
24427,şüphesi,[kimsenin şüphesi yok]
24428,şüphesiz,"[beni burada mı, hiç şüphesiz ki, şüphesiz ki ..."
24429,şüpheye,[şüpheye yer bırakmayacak]
24430,şık,"[çok şık bir, şık bir yer, ne kadar şık, şık b..."


In [29]:
df_word_in_three["word"].nunique()

24432

In [30]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [31]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_four

,word,fourgram
0,bir,bir şey var mı
1,bir,bir sorun mu var
2,bir,gereken bir şey var
3,bir,başka bir şey var
4,bir,bir şey daha var
...,...,...
1028631,ayrıntısına,en ince ayrıntısına kadar
1028632,ayrıntısına,en ufak ayrıntısına kadar
1028633,ayrıntısına,şeyi en ince ayrıntısına
1028634,kardeşisin,de onun kız kardeşisin


In [32]:
df_word_in_four_list = df_word_in_four.groupby(["word"])["fourgram"].apply(list).reset_index(name="fourgram")
df_word_in_four_list

,word,fourgram
0,a,"[a b c d, i t a r, g i t a, a r i s, t a r i, ..."
1,aa,"[aa şey aa yani, ah aa bilirsin şey, aa yani t..."
2,aaa,"[aaa yoksa bu öyle, benimle aaa evlenir misin,..."
3,aaaah,[aaaah sadece bir saniyeliğine]
4,aaah,"[hiç aaah diye bağırıp, aaah diye bağırıp havaya]"
...,...,...
34299,şüphesiz,"[şüphesiz ki var bir, hiç şüphesiz bu bir, şüp..."
34300,şüpheye,"[en ufak bir şüpheye, parkın güvenliği hakkınd..."
34301,şüpheyle,[şüpheyle yaklaşıyorum tabii kendi]
34302,şık,"[ne kadar da şık, bu çok şık bir, burası çok ş..."


In [33]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [34]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_five

,word,fivegram
0,güvenli,burası senin için güvenli değil
1,güvenli,bunun güvenli olduğundan emin misin
2,güvenli,onu güvenli bir yere götür
3,güvenli,seni güvenli bir yere götüreceğim
4,güvenli,bunun güvenli olduğuna emin misin
...,...,...
474808,açılışını,aynı drama açılışını defalarca kez
474809,batıyoruz,batıyoruz ve gücümüz de yok
474810,ayrıntısına,şeyi en ince ayrıntısına kadar
474811,ayrıntısına,her şeyi en ince ayrıntısına


In [35]:
df_word_in_five_list = df_word_in_five.groupby(["word"])["fivegram"].apply(list).reset_index(name="fivegram")
df_word_in_five_list

,word,fivegram
0,a,"[take a woman like you, i t a r i, t a r i s, ..."
1,aa,"[şey aa yani tüm kasabaya, şey aa onlar yani ş..."
2,aaa,"[aaa yoksa bu öyle bir, aaa şey çünkü daha faz..."
3,aaah,"[hiç aaah diye bağırıp havaya, aaah diye bağır..."
4,aah,"[aah en uzun yolculuğuna daha, aah o gün olanl..."
...,...,...
27035,şüphesi,"[bundan hiç şüphesi oldu mu, hiç şüphesi oldu ..."
27036,şüphesiz,"[şüphesiz ki var bir nedeni, şüphesiz beynimde..."
27037,şüpheye,"[parkın güvenliği hakkında şüpheye düştü, oldu..."
27038,şık,"[şık bir isim değil mi, burası çok şık bir res..."


In [36]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [37]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_six

,word,sixgram
0,bebeğimiz,söylesene birlikte bebeğimiz olsun istiyor musun
1,bebeğimiz,bebeğimiz aynen böyle bir şey olacak
2,bebeğimiz,bebeğimiz hakkında böyle konuşmana çok şaşırdım
3,bebeğimiz,küçük bebeğimiz hakkında böyle konuşmana çok
4,bebeğimiz,asıl sorun şu ki bebeğimiz olamıyor
...,...,...
1285174,aştığını,bu tarz küçük duyguları aştığını düşünüyordum
1285175,aştığını,aştığını bilerek yüzümde bir gülümsemeyi zorlu...
1285176,incitmiş,onu incitmiş o da öç almak
1285177,incitmiş,birisi onu incitmiş o da öç


In [38]:
df_word_in_six_list = df_word_in_six.groupby(["word"])["sixgram"].apply(list).reset_index(name="sixgram")
df_word_in_six_list

,word,sixgram
0,a,"[a r i s y e, i t a r i s, t a r i s y, g i t ..."
1,aa,"[yani aa ahbap ya da adamım, oh bu sorun değil..."
2,aaa,"[aaa yoksa bu öyle bir resim, aaa şey buna ger..."
3,aaaah,[aaaah sadece bir saniyeliğine gözlerini açtı]
4,aaah,"[hiç aaah diye bağırıp havaya ateş, hayır hiç ..."
...,...,...
39759,şüpheyle,"[nasıl şüpheyle baktığını biliyorsun değil mi,..."
39760,şık,"[ama ikinci bölüm gerçekten çok şık, q bölümü ..."
39761,şıllık,[geri dön seni şişko sakallı şıllık]
39762,şımarık,"[aynen öyle kır hepsini şımarık gine, öyle kır..."


In [39]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [40]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_seven

,word,sevengram
0,vermiştin,sen en son ne zaman ara vermiştin
1,vermiştin,normal bir insan gibi çalışmaya söz vermiştin
2,vermiştin,floridaya varınca bu işi bitirmeye söz vermiştin
3,vermiştin,aptalca bir şey yapmayacağına dair söz vermiştin
4,vermiştin,ama hani bana geçen sefer söz vermiştin
...,...,...
774075,yollayalım,için hava gücünün geri kalanını da yollayalım
774076,yollayalım,vücuduna bir kurşun sıkıp size cesedini yollay...
774077,aştığını,oldukça aştığını bilerek yüzümde bir gülümseme...
774078,incitmiş,onu incitmiş o da öç almak istiyor


In [41]:
df_word_in_seven_list = df_word_in_seven.groupby(["word"])["sevengram"].apply(list).reset_index(name="sevengram")
df_word_in_seven_list

,word,sevengram
0,a,"[i t a r i s y, t a r i s y e, a r i s y e n, ..."
1,aa,"[ve o aa şey aa yani tüm, aa bu resim bayan re..."
2,aaa,"[aaa yoksa bu öyle bir resim mi, aaa şey çünkü..."
3,aaah,"[aaah diye bağırıp havaya ateş ettin mi, hiç a..."
4,aah,"[aah en uzun yolculuğuna daha yeni başladın, a..."
...,...,...
35615,şüphesiz,[gizli görevde temas kurduğun kişilerin bir li...
35616,şüpheye,[olduğum yerden gördüğüm kadarıyla şüpheye hiç...
35617,şüpheyle,[bize nasıl şüpheyle baktığını biliyorsun deği...
35618,şık,"[iş bulur bulmaz seni şehrin en şık, bulmaz se..."


In [42]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [43]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_eight

,word,eightgram
0,bir,bir iki üç dört beş altı yedi sekiz
1,bir,sekiz yedi altı beş dört üç iki bir
2,bir,sizin için yapabileceğim başka bir şey var mı
3,bir,neşeli ruhu bir gün yelken açtı ta denizden
4,bir,giden genç kadın için bana bir şarkı söyle
...,...,...
696078,yollayalım,iyisi vücuduna bir kurşun sıkıp size cesedini ...
696079,ayrıntısına,en ince ayrıntısına kadar bilen biri polisten ...
696080,ayrıntısına,ince ayrıntısına kadar bilen biri polisten bir...
696081,incitmiş,birisi onu incitmiş o da öç almak istiyor


In [44]:
df_word_in_eight_list = df_word_in_eight.groupby(["word"])["eightgram"].apply(list).reset_index(name="eightgram")
df_word_in_eight_list

,word,eightgram
0,a,"[i t a r i s y e, t a r i s y e n, g i t a r i..."
1,aa,"[ve o aa şey aa yani tüm kasabaya, kurbağa gib..."
2,aaa,"[aaa o da sütten çıkmış ak kaşık öyle, aaa eğe..."
3,aaah,[hayır hiç aaah diye bağırıp havaya ateş etmed...
4,aah,[lana aah bundan biraz daha karmaşık biliyor m...
...,...,...
35369,şüphesiz,"[hiç şüphesiz ki bryan o bir baş belası, şüphe..."
35370,şüpheye,[olduğum yerden gördüğüm kadarıyla şüpheye hiç...
35371,şık,[iş bulur bulmaz seni şehrin en şık restoranın...
35372,şımarık,[seninle evlenirken bencil ve şımarık biri old...


In [45]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [46]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_nine

,word,ninegram
0,bildirin,dünyaya neler olduğunu bildirin hem de bunu de...
1,öpmek,beni öpmek zorunda değilsin ama ben seni öpmek...
2,öpmek,yani ahbap onu öpmek istiyorsan hiç durma ve öp
3,öpmek,ahbap onu öpmek istiyorsan hiç durma ve öp onu
4,öpmek,sonra beni öpmek için ağzını öyle bir açtı ki
...,...,...
319895,uygulanabilir,yaşına ve hastalığının ilerleme hızına bakarsa...
319896,uygulanabilir,ve hastalığının ilerleme hızına bakarsak bu du...
319897,yollayalım,ülkemizin çıkarları için hava gücünün geri kal...
319898,yollayalım,en iyisi vücuduna bir kurşun sıkıp size cesedi...


In [47]:
df_word_in_nine_list = df_word_in_nine.groupby(["word"])["ninegram"].apply(list).reset_index(name="ninegram")
df_word_in_nine_list

,word,ninegram
0,a,"[i t a r i s y e n, g i t a r i s y e, e g o m..."
1,aa,[şey bu kadar basit olmayabilir belkide yani a...
2,aaa,"[aaa o da sütten çıkmış ak kaşık öyle mi, aaa ..."
3,aah,[oh uh evet uh ben buradayım saygıdeğer efendi...
4,abartılı,[ve sanıyorum onların seni sevdiği kısmı biraz...
...,...,...
27724,şüphesiz,[şüphesiz ki yapamazsan daha başlamadan yenilg...
27725,şüpheye,[en ufak bir şüpheye yer olmadan birini böyles...
27726,şık,[iş bulur bulmaz seni şehrin en şık restoranın...
27727,şımarık,[siz güvensiz şımarık ve erkeklerin çekici bul...


In [48]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [49]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_ten

,word,tengram
0,etsen,acele etsen iyi olur çünkü o bizi teker teker ...
1,etsen,ismini falan vermeden rapor etsen yeterli olur...
2,etsen,kabul etsen de etmesen de buraya bir şey için ...
3,etsen,laflarına dikkat etsen iyi edersin yoksa başka...
4,etsen,şimdi terk etsen bile beni sahip olduğum tek ş...
...,...,...
272885,wexler,ozzie wexler adlı bir bilim adamı bugün alışve...
272886,uygulanabilir,yaşına ve hastalığının ilerleme hızına bakarsa...
272887,uygulanabilir,gurur ve utanç çalışanlara karşı değil üzerind...
272888,kardeşisin,kralın kardeşisin demek ona rağmen ordusuna se...


In [50]:
df_word_in_ten_list = df_word_in_ten.groupby(["word"])["tengram"].apply(list).reset_index(name="tengram")
df_word_in_ten_list

,word,tengram
0,a,"[g i t a r i s y e n, a takımı değil belki ama..."
1,aa,[sinek yakalayan kurbağa gibi dilini dışarı çı...
2,aah,[oh uh evet uh ben buradayım saygıdeğer efendi...
3,ab,[genç kız tedavi edici ab için gayet sağlıklı ...
4,abartmış,[halk tipi imajını abartmış ama istese tırnakl...
...,...,...
28380,şüphesi,[bundan kimsenin şüphesi yok ama burada çiftli...
28381,şüphesiz,[şüphesiz erdemli bir lider sadece küçük bir a...
28382,şüpheye,[onun beni şüpheye yer vermeyecek şekilde gözl...
28383,şık,[bir iş bulur bulmaz seni şehrin en şık restor...


In [51]:
dfs = [df_word_select, df_word_in_two_list, df_word_in_three_list, df_word_in_four_list, df_word_in_five_list, df_word_in_six_list, df_word_in_seven_list, df_word_in_eight_list, df_word_in_nine_list, df_word_in_ten_list]

In [52]:
df_word_merge = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='left'), dfs)
df_word_merge

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir,18835735,"[bir şey, bu bir, bir dakika, bir şeyler, başk...","[bir şey yok, bir şey var, başka bir şey, bir ...","[bir şey var mı, bir sorun mu var, gereken bir...","[zaman yolculuğunu icat ettiğim gündür, sana b...","[sana söylemem gereken bir şey var, bir iki üç...",[ne kadar özel olduğunu başkasının anlayacağın...,"[bir iki üç dört beş altı yedi sekiz, sekiz ye...","[dokuz sekiz yedi altı beş dört üç iki bir, bi...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
1,bu,11062659,"[bu kadar, bu da, bu çok, bu bir, bu ne, bu yü...","[bu da ne, neden bu kadar, bu doğru değil, bu ...","[bu da ne demek, bu da ne böyle, bu iyi bir fi...","[bu da ne demek oluyor, bu da ne demek şimdi, ...","[bu iyi bir şey değil mi, bence bu iyi bir fik...","[bu sana bir şey ifade ediyor mu, evime varmay...","[bu sabah uyandın bütün o sevgi yok olmuş, bu ...",[bu gün bir kez daha biri bir yerlerde ağlayac...,[bu işlemi bu akşam başlatmak istiyorsak yapac...
2,ne,8025880,"[ne oldu, ne kadar, ne yapıyorsun, ne demek, n...","[bu da ne, ne demek istiyorsun, ne işin var, n...","[burada ne işin var, bu da ne demek, en son ne...","[bu da ne demek oluyor, bu da ne demek şimdi, ...","[onu en son ne zaman gördün, bunun ne anlama g...","[ne kadar az bilirsen o kadar iyi, kadınların ...",[bunun senin için ne kadar zor olduğunu biliyo...,[efendi luke sizi tekrar işler vaziyette görme...,[ne dediğimi duymadın mı evlat ben de ona evet...
3,ve,7766036,"[ve ben, ve bu, ve bir, sen ve, ve sen, ve o, ...","[sen ve ben, bayanlar ve baylar, ve ben de, ve...","[sadece sen ve ben, oğul ve kutsal ruh, baba o...","[baba oğul ve kutsal ruh, oğul ve kutsal ruh a...","[baba oğul ve kutsal ruh adına, evrenin her ya...","[bir ve iki ve üç ve dört, ve sen herkesin sen...","[ve sen herkesin seni tanıdığı o yerde olmak, ...",[ve sen herkesin seni tanıdığı o yerde olmak i...,[kız artık bizim için bir önem arz etmiyor mu ...
4,için,5484109,"[senin için, benim için, için bir, bunun için,...","[için özür dilerim, için teşekkür ederim, beni...","[sizin için ne yapabilirim, senin için ne yapa...","[rahatsız ettiğim için özür dilerim, senin içi...","[için yapabileceğim bir şey var mı, benim için...","[senin için yapabileceğim bir şey var mı, sizi...",[sizin için yapabileceğim başka bir şey var mı...,[buralara kadar gelebilmek için pek çok duvarı...,[annem bana bazı şeyleri anlatmak için mutlaka...
...,...,...,...,...,...,...,...,...,...,...,...
44995,hayırbu,871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44996,konferansına,871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44997,kardeşisin,871,"[onun kardeşisin, kız kardeşisin]","[kardeşisin değil mi, onun kız kardeşisin]","[de onun kız kardeşisin, kız kardeşisin değil mi]",[sen de onun kız kardeşisin],NaN,NaN,NaN,NaN,[kralın kardeşisin demek ona rağmen ordusuna s...
44998,kaldırıyorsun,871,[kadar kaldırıyorsun],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#df_word_merge.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx", index=False)
df_word_merge.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv", index=False)

In [54]:
#df_word_text = pd.read_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx")
df_word_text = pd.read_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv")

In [55]:
df_word_merge2 = df_word_text[~(df_word_text["twogram"].isnull() & df_word_text["threegram"].isnull() & df_word_text["fourgram"].isnull() & df_word_text["fivegram"].isnull() & \
df_word_text["sixgram"].isnull() & df_word_text["sevengram"].isnull() & df_word_text["eightgram"].isnull() & df_word_text["ninegram"].isnull() & df_word_text["tengram"].isnull())]
df_word_merge2

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir,18835735,"['bir şey', 'bu bir', 'bir dakika', 'bir şeyle...","['bir şey yok', 'bir şey var', 'başka bir şey'...","['bir şey var mı', 'bir sorun mu var', 'gereke...","['zaman yolculuğunu icat ettiğim gündür', 'san...","['sana söylemem gereken bir şey var', 'bir iki...",['ne kadar özel olduğunu başkasının anlayacağı...,"['bir iki üç dört beş altı yedi sekiz', 'sekiz...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,bu,11062659,"['bu kadar', 'bu da', 'bu çok', 'bu bir', 'bu ...","['bu da ne', 'neden bu kadar', 'bu doğru değil...","['bu da ne demek', 'bu da ne böyle', 'bu iyi b...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu iyi bir şey değil mi', 'bence bu iyi bir ...","['bu sana bir şey ifade ediyor mu', 'evime var...","['bu sabah uyandın bütün o sevgi yok olmuş', '...",['bu gün bir kez daha biri bir yerlerde ağlaya...,['bu işlemi bu akşam başlatmak istiyorsak yapa...
2,ne,8025880,"['ne oldu', 'ne kadar', 'ne yapıyorsun', 'ne d...","['bu da ne', 'ne demek istiyorsun', 'ne işin v...","['burada ne işin var', 'bu da ne demek', 'en s...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['onu en son ne zaman gördün', 'bunun ne anlam...","['ne kadar az bilirsen o kadar iyi', 'kadınlar...",['bunun senin için ne kadar zor olduğunu biliy...,['efendi luke sizi tekrar işler vaziyette görm...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,ve,7766036,"['ve ben', 've bu', 've bir', 'sen ve', 've se...","['sen ve ben', 'bayanlar ve baylar', 've ben d...","['sadece sen ve ben', 'oğul ve kutsal ruh', 'b...","['baba oğul ve kutsal ruh', 'oğul ve kutsal ru...","['baba oğul ve kutsal ruh adına', 'evrenin her...","['bir ve iki ve üç ve dört', 've sen herkesin ...",['ve sen herkesin seni tanıdığı o yerde olmak'...,['ve sen herkesin seni tanıdığı o yerde olmak ...,['kız artık bizim için bir önem arz etmiyor mu...
4,için,5484109,"['senin için', 'benim için', 'için bir', 'bunu...","['için özür dilerim', 'için teşekkür ederim', ...","['sizin için ne yapabilirim', 'senin için ne y...","['rahatsız ettiğim için özür dilerim', 'senin ...","['için yapabileceğim bir şey var mı', 'benim i...","['senin için yapabileceğim bir şey var mı', 's...",['sizin için yapabileceğim başka bir şey var m...,['buralara kadar gelebilmek için pek çok duvar...,['annem bana bazı şeyleri anlatmak için mutlak...
...,...,...,...,...,...,...,...,...,...,...,...
44993,aştığını,871,NaN,NaN,NaN,NaN,['oldukça aştığını bilerek yüzümde bir gülümse...,['oldukça aştığını bilerek yüzümde bir gülümse...,NaN,NaN,NaN
44994,incitmiş,871,NaN,NaN,NaN,NaN,"['onu incitmiş o da öç almak', 'birisi onu inc...","['onu incitmiş o da öç almak istiyor', 'birisi...","['birisi onu incitmiş o da öç almak istiyor', ...",NaN,NaN
44997,kardeşisin,871,"['onun kardeşisin', 'kız kardeşisin']","['kardeşisin değil mi', 'onun kız kardeşisin']","['de onun kız kardeşisin', 'kız kardeşisin değ...",['sen de onun kız kardeşisin'],NaN,NaN,NaN,NaN,['kralın kardeşisin demek ona rağmen ordusuna ...
44998,kaldırıyorsun,871,['kadar kaldırıyorsun'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
#df_word_merge2.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)
df_word_merge2.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.csv", index=False)

##### Twogram

In [ ]:
# 2 Way Fast
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_three = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three

In [ ]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_four

In [ ]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_five

In [ ]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_six

In [ ]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_seven

In [ ]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_eight

In [ ]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_nine

In [ ]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_ten

In [ ]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

In [ ]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

In [ ]:
#df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx", index=False)
df_twogram_merge.to_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_twogram_text = pd.read_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx")
df_twogram_text = pd.read_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.csv")

In [ ]:
df_twogram_merge2 = df_twogram_text[~(df_twogram_text["threegram"].isnull() & df_twogram_text["fourgram"].isnull() & df_twogram_text["fivegram"].isnull() & df_twogram_text["sixgram"].isnull() & \
df_twogram_text["sevengram"].isnull() & df_twogram_text["eightgram"].isnull() & df_twogram_text["ninegram"].isnull() & df_twogram_text["tengram"].isnull())]
df_twogram_merge2

In [ ]:
#df_twogram_merge2.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)
df_twogram_merge2.to_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.csv", index=False)

##### Threegram

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_four

In [ ]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_five

In [ ]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_six

In [ ]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_seven

In [ ]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_eight

In [ ]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_nine

In [ ]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_ten

In [ ]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

In [ ]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [ ]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

In [ ]:
#df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx", index=False)
df_threegram_merge.to_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_threegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx")
df_threegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.csv")

In [ ]:
df_threegram_merge2 = df_threegram_text[~(df_threegram_text["fourgram"].isnull() & df_threegram_text["fivegram"].isnull() & df_threegram_text["sixgram"].isnull() & \
df_threegram_text["sevengram"].isnull() & df_threegram_text["eightgram"].isnull() & df_threegram_text["ninegram"].isnull() & df_threegram_text["tengram"].isnull())]
df_threegram_merge2

In [ ]:
#df_threegram_merge2.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)
df_threegram_merge2.to_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.csv", index=False)

##### Fourgram

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
#df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx", index=False)
df_fourgram_merge.to_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_fourgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx")
df_fourgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.csv")

In [ ]:
df_fourgram_merge2 = df_fourgram_text[~(df_fourgram_text["fivegram"].isnull() & df_fourgram_text["sixgram"].isnull() & df_fourgram_text["sevengram"].isnull() & \
df_fourgram_text["eightgram"].isnull() & df_fourgram_text["ninegram"].isnull() & df_fourgram_text["tengram"].isnull())]
df_fourgram_merge2

In [ ]:
#df_fourgram_merge2.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)
df_fourgram_merge2.to_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.csv", index=False)

##### Fivegram

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
#df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx", index=False)
df_fivegram_merge.to_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_fivegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx")
df_fivegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.csv")

In [ ]:
df_fivegram_merge2 = df_fivegram_text[~(df_fivegram_text["sixgram"].isnull() & df_fivegram_text["sevengram"].isnull() & \
df_fivegram_text["eightgram"].isnull() & df_fivegram_text["ninegram"].isnull() & df_fivegram_text["tengram"].isnull())]
df_fivegram_merge2

In [ ]:
#df_fivegram_merge2.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)
df_fivegram_merge2.to_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.csv", index=False)

##### Sixgram

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
#df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx", index=False)
df_sixgram_merge.to_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_sixgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx")
df_sixgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.csv")

In [ ]:
df_sixgram_merge2 = df_sixgram_text[~(df_sixgram_text["sevengram"].isnull() & df_sixgram_text["eightgram"].isnull() & df_sixgram_text["ninegram"].isnull() & df_sixgram_text["tengram"].isnull())]
df_sixgram_merge2

In [ ]:
#df_sixgram_merge2.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)
df_sixgram_merge2.to_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.csv", index=False)

##### Sevengram

In [20]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [21]:
search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_eight

,sevengram,eightgram
0,dobby bundan ancak efendisi ona giyecek verirse,dobby bundan ancak efendisi ona giyecek verirs...
1,ve son katılan genç anakin skywalker yerli,ve son katılan genç anakin skywalker yerli bir
2,yönetici oldum bir sürü siyasi yardım alıyorum,artık yönetici oldum bir sürü siyasi yardım al...
3,size onların günlük hayatı hakkında biraz bilgi,size onların günlük hayatı hakkında biraz bilg...
4,size onların günlük hayatı hakkında biraz bilgi,tablo size onların günlük hayatı hakkında bira...
...,...,...
125750,görücü usulü evlenmek daha çok işe yarıyor,bazen görücü usulü evlenmek daha çok işe yarıyor
125751,konuda çok yanlış olduğundan kesinlikle eminim...,bu konuda çok yanlış olduğundan kesinlikle emi...
125752,devlet başkanı belki yarın gelir ama bunlar,devlet başkanı belki yarın gelir ama bunlar ço...
125753,sen sevgilimi çaldın ve onunla iki hafta,sen sevgilimi çaldın ve onunla iki hafta gününü


In [22]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

,sevengram,eightgram
0,a annem dedi ki seni teşvik etmek,"[hepsi a annem dedi ki seni teşvik etmek, a an..."
1,a b c d diye alfabetik gider,[alanları a b c d diye alfabetik gider]
2,a e d a l u s,[d a e d a l u s]
3,a evet şu güney sokaktaki arabaları çektikleri,[a evet şu güney sokaktaki arabaları çektikler...
4,a n d o r u m,[p a n d o r u m]
...,...,...
101897,şüphesiz ki yapamazsan daha başlamadan yenilgi...,[şüphesiz ki yapamazsan daha başlamadan yenilg...
101898,şüphesiz o da herkes gibi bir avukat,[şüphesiz o da herkes gibi bir avukat alt]
101899,şüphesiz şimdiye kadar gördüğüm en kötü korsan,[bu şüphesiz şimdiye kadar gördüğüm en kötü ko...
101900,şık bir şeyler gir ve elini çabuk,"[şık bir şeyler gir ve elini çabuk tut, üstüne..."


In [23]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [24]:
search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_nine

,sevengram,ninegram
0,yaşamak varken neden korkunç bir şekilde can,bana katılıp yaşamak varken neden korkunç bir ...
1,bu kadar güzel bir haber almadan geçiriyorlar,tüm hayatlarını bu kadar güzel bir haber almad...
2,ve son katılan genç anakin skywalker yerli,ve son katılan genç anakin skywalker yerli bir...
3,size onların günlük hayatı hakkında biraz bilgi,bu tablo size onların günlük hayatı hakkında b...
4,size onların günlük hayatı hakkında biraz bilgi,tablo size onların günlük hayatı hakkında bira...
...,...,...
70703,özel olarak çünkü bu sadece bir dosta,özel olarak çünkü bu sadece bir dosta söylenec...
70704,akşam üçüncü ve muhtemelen son olarak yerde,atom bu akşam üçüncü ve muhtemelen son olarak ...
70705,herifi durdurmak için sadece bir saat kırk,bu herifi durdurmak için sadece bir saat kırk ...
70706,herifi durdurmak için sadece bir saat kırk,herifi durdurmak için sadece bir saat kırk dak...


In [25]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

,sevengram,ninegram
0,a annem dedi ki seni teşvik etmek,[hepsi a annem dedi ki seni teşvik etmek lazım]
1,a b c d diye alfabetik gider,[park alanları a b c d diye alfabetik gider]
2,a onlardan biri olabilir ve b hala,[çünkü a onlardan biri olabilir ve b hala kızg...
3,a r i s y e n,[i t a r i s y e n]
4,a takımı değil belki ama elimizden gelenin,[a takımı değil belki ama elimizden gelenin en...
...,...,...
58700,şüpheliyi suç mahalline getir ve kendini ele,[şüpheliyi suç mahalline getir ve kendini ele ...
58701,şüphen varsa beni vur parayı al ve,[şüphen varsa beni vur parayı al ve güzelce uy...
58702,şüphesiz ki yapamazsan daha başlamadan yenilgi...,[şüphesiz ki yapamazsan daha başlamadan yenilg...
58703,şüphesiz o da herkes gibi bir avukat,[şüphesiz o da herkes gibi bir avukat alt edeb...


In [26]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [27]:
search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_ten

,sevengram,tengram
0,bu kadar güzel bir haber almadan geçiriyorlar,bazıları tüm hayatlarını bu kadar güzel bir ha...
1,ama size bu bilgiyi vermem kurallara aykırı,ne dediğinizi anlıyorum ama size bu bilgiyi ve...
2,size onların günlük hayatı hakkında biraz bilgi,bu tablo size onların günlük hayatı hakkında b...
3,bir iki üç dört beş altı yedi,bir iki üç dört beş altı yedi sekiz dokuz on
4,bir iki üç dört beş altı yedi,bir iki üç dört beş altı yedi sekiz dokuz dön
...,...,...
24352,sanıyor ve beni de en yakın arkadaşı,onu kendi oğlu sanıyor ve beni de en yakın ark...
24353,aileyi kadını çocukları herkesi pizza ısmarlar...,adam bütün aileyi kadını çocukları herkesi piz...
24354,ben buradayken gelecekte aileme yardım edeceği...,yaptıklarımın bir şekilde ben buradayken gelec...
24355,bildiğimiz kadarıyla da altı ay önce irakta,bizim bildiğimiz kadarıyla da altı ay önce ira...


In [28]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

,sevengram,tengram
0,a onlardan biri olabilir ve b hala,[çünkü a onlardan biri olabilir ve b hala kızg...
1,a r i s y e n,[g i t a r i s y e n]
2,a takımı değil belki ama elimizden gelenin,[a takımı değil belki ama elimizden gelenin en...
3,abc gündüz ve gece için yeni evli,[abc gündüz ve gece için yeni evli oyunu nu se...
4,abd hava üssünde patlayacak atom bombası daha,[abd hava üssünde patlayacak atom bombası daha...
...,...,...
24198,şöyle söyleyeyim sanki şu an burada dikiliyor,[şöyle söyleyeyim sanki şu an burada dikiliyor...
24199,şöyle yeşil nakite ihtiyacı vardır anlıyorsun ya,[bilirsin bir kardeş şöyle yeşil nakite ihtiya...
24200,şükürler olsun ki o zamandan bu yana,[şükürler olsun ki o zamandan bu yana uzun yol...
24201,şüpheli birinin beyin ameliyatı yapmasına izin...,[elbette siyasi olarak şüpheli birinin beyin a...


In [29]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [30]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

,sevengram,frequency,eightgram,ninegram,tengram
0,bir iki üç dört beş altı yedi,1455,"[bir iki üç dört beş altı yedi sekiz, ve bir i...","[bir iki üç dört beş altı yedi sekiz dokuz, ve...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
1,iki üç dört beş altı yedi sekiz,1080,"[bir iki üç dört beş altı yedi sekiz, iki üç d...","[bir iki üç dört beş altı yedi sekiz dokuz, ik...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
2,bana söylemek istediğin bir şey var mı,752,"[önce bana söylemek istediğin bir şey var mı, ...",[çıkmadan önce bana söylemek istediğin bir şey...,[arabadan çıkmadan önce bana söylemek istediği...
3,yedi altı beş dört üç iki bir,596,"[sekiz yedi altı beş dört üç iki bir, yedi alt...","[dokuz sekiz yedi altı beş dört üç iki bir, se...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
4,sekiz yedi altı beş dört üç iki,551,"[dokuz sekiz yedi altı beş dört üç iki, sekiz ...","[dokuz sekiz yedi altı beş dört üç iki bir, on...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
...,...,...,...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9,[bu akşam üçüncü ve muhtemelen son olarak yerde],[atom bu akşam üçüncü ve muhtemelen son olarak...,NaN
141655,herifi durdurmak için sadece bir saat kırk,9,[herifi durdurmak için sadece bir saat kırk da...,[bu herifi durdurmak için sadece bir saat kırk...,[bu herifi durdurmak için sadece bir saat kırk...
141656,görücü usulü evlenmek daha çok işe yarıyor,9,[bazen görücü usulü evlenmek daha çok işe yarı...,NaN,NaN
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9,[bu konuda çok yanlış olduğundan kesinlikle em...,NaN,NaN


In [31]:
#df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx", index=False)
df_sevengram_merge.to_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.csv", index=False)

In [32]:
#df_sevengram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx")
df_sevengram_text = pd.read_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.csv")

In [33]:
df_sevengram_merge2 = df_sevengram_text[~(df_sevengram_text["eightgram"].isnull() & df_sevengram_text["ninegram"].isnull() & df_sevengram_text["tengram"].isnull())]
df_sevengram_merge2

,sevengram,frequency,eightgram,ninegram,tengram
0,bir iki üç dört beş altı yedi,1455,"['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,iki üç dört beş altı yedi sekiz,1080,"['bir iki üç dört beş altı yedi sekiz', 'iki ü...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,bana söylemek istediğin bir şey var mı,752,['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,yedi altı beş dört üç iki bir,596,"['sekiz yedi altı beş dört üç iki bir', 'yedi ...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
4,sekiz yedi altı beş dört üç iki,551,"['dokuz sekiz yedi altı beş dört üç iki', 'sek...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
...,...,...,...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9,['bu akşam üçüncü ve muhtemelen son olarak yer...,['atom bu akşam üçüncü ve muhtemelen son olara...,NaN
141655,herifi durdurmak için sadece bir saat kırk,9,['herifi durdurmak için sadece bir saat kırk d...,['bu herifi durdurmak için sadece bir saat kır...,['bu herifi durdurmak için sadece bir saat kır...
141656,görücü usulü evlenmek daha çok işe yarıyor,9,['bazen görücü usulü evlenmek daha çok işe yar...,NaN,NaN
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9,['bu konuda çok yanlış olduğundan kesinlikle e...,NaN,NaN


In [34]:
#df_sevengram_merge2.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)
df_sevengram_merge2.to_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.csv", index=False)

##### Eightgram

In [35]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [36]:
search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_eight_in_nine

,eightgram,ninegram
0,ve eğer o çingene bir zarar daha verirse,ve eğer o çingene bir zarar daha verirse sen
1,tür bir oyun olduğunu öğrenmek istiyorsun nası...,ne tür bir oyun olduğunu öğrenmek istiyorsun n...
2,tür bir oyun olduğunu öğrenmek istiyorsun nası...,tür bir oyun olduğunu öğrenmek istiyorsun nası...
3,hey sizin istediğiniz bildik yollar öyle yapal...,hey sizin istediğiniz bildik yollar öyle yapal...
4,bir şekilde o pislik hala yaşıyordu ve bana,bir şekilde o pislik hala yaşıyordu ve bana ba...
...,...,...
80530,hong konga gidip bütün çeteyi temizleyecek ces...,hong konga gidip bütün çeteyi temizleyecek ces...
80531,ülkenin beni ne duruma getirdiğini bir tek sen,bu ülkenin beni ne duruma getirdiğini bir tek sen
80532,ülkenin beni ne duruma getirdiğini bir tek sen,ülkenin beni ne duruma getirdiğini bir tek sen...
80533,bayan şu olaylar çerçevesinde kocanızın sandığ...,bayan şu olaylar çerçevesinde kocanızın sandığ...


In [37]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

,eightgram,ninegram
0,a annem dedi ki seni teşvik etmek lazım,[hepsi a annem dedi ki seni teşvik etmek lazım]
1,a kenarı b kenarı ve c kenarına eşittir,[üçgeni a kenarı b kenarı ve c kenarına eşittir]
2,a noktasından b noktasına olabildiğince hızlı ...,[a noktasından b noktasına olabildiğince hızlı...
3,a onlardan biri olabilir ve b hala kızgın,[çünkü a onlardan biri olabilir ve b hala kızg...
4,a takımı değil belki ama elimizden gelenin en,[a takımı değil belki ama elimizden gelenin en...
...,...,...
70432,şüphesiz ki yapamazsan daha başlamadan yenilgi...,[şüphesiz ki yapamazsan daha başlamadan yenilg...
70433,şüphesiz o da herkes gibi bir avukat alt,[şüphesiz o da herkes gibi bir avukat alt edeb...
70434,şüphesiz tek başına bu bile çok büyük bir,[şüphesiz tek başına bu bile çok büyük bir suç...
70435,şüpheye yer vermeyecek şekilde gözlem altına a...,[beni şüpheye yer vermeyecek şekilde gözlem al...


In [38]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [39]:
search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_eight_in_ten

,eightgram,tengram
0,bir farenin duvardan geçtiği gibi kapının altı...,bir farenin duvardan geçtiği gibi kapının altı...
1,herkesten daha iyi biliyorsun ki memeler kimse...,buradaki herkesten daha iyi biliyorsun ki meme...
2,bir sürü şey için söz veriyorsun ama hiçbirini,bana bir sürü şey için söz veriyorsun ama hiçb...
3,ben bunu bana yapacağını ya da yapabileceğini hiç,ama ben bunu bana yapacağını ya da yapabileceğ...
4,bir iki üç dört beş altı yedi sekiz,bir iki üç dört beş altı yedi sekiz dokuz on
...,...,...
31027,baylar brooklyne gidecek bir sonraki tren varm...,bayanlar ve baylar brooklyne gidecek bir sonra...
31028,sabahlar seni olduğunu sandığın gibi değil old...,çünkü sabahlar seni olduğunu sandığın gibi değ...
31029,bu ofisin başına geçmeyi kabul ettiğimde bu işi,bu ofisin başına geçmeyi kabul ettiğimde bu iş...
31030,o maymunlar ordusuna katıldın çünkü kendin içi...,o maymunlar ordusuna katıldın çünkü kendin içi...


In [40]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

,eightgram,tengram
0,a kenarı b kenarı ve c kenarına eşittir,[abc üçgeni a kenarı b kenarı ve c kenarına eş...
1,a onlardan biri olabilir ve b hala kızgın,[çünkü a onlardan biri olabilir ve b hala kızg...
2,a takımı değil belki ama elimizden gelenin en,[a takımı değil belki ama elimizden gelenin en...
3,abc üçgeni a kenarı b kenarı ve c,[abc üçgeni a kenarı b kenarı ve c kenarına eş...
4,abd hava üssünde patlayacak atom bombası daha çok,[abd hava üssünde patlayacak atom bombası daha...
...,...,...
30966,şükürler olsun ki bu kiliseyi koruması için bi...,[tanrıya şükürler olsun ki bu kiliseyi korumas...
30967,şükürler olsun ki o zamandan bu yana uzun,[şükürler olsun ki o zamandan bu yana uzun yol...
30968,şüphen varsa beni vur parayı al ve güzelce,[bundan şüphen varsa beni vur parayı al ve güz...
30969,şüphesiz erdemli bir lider sadece küçük bir ar...,[şüphesiz erdemli bir lider sadece küçük bir a...


In [41]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [42]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

,eightgram,frequency,ninegram,tengram
0,bir iki üç dört beş altı yedi sekiz,1060,"[bir iki üç dört beş altı yedi sekiz dokuz, ve...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
1,dokuz sekiz yedi altı beş dört üç iki,513,"[dokuz sekiz yedi altı beş dört üç iki bir, on...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
2,sekiz yedi altı beş dört üç iki bir,497,"[dokuz sekiz yedi altı beş dört üç iki bir, se...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
3,iki üç dört beş altı yedi sekiz dokuz,411,"[bir iki üç dört beş altı yedi sekiz dokuz, ik...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
4,üç dört beş altı yedi sekiz dokuz on,329,"[iki üç dört beş altı yedi sekiz dokuz on, üç ...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
...,...,...,...,...
109469,ve bunu her yaptığında takım arkadaşlarından b...,7,NaN,NaN
109470,gizli gizli dolaşan genç bir haham imajı canla...,7,NaN,NaN
109471,için tek yol kahrolası paramı bana geri vermen,7,NaN,NaN
109472,teşkilatın görev kapsamında olsa bile görev se...,7,NaN,NaN


In [43]:
#df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx", index=False)
df_eightgram_merge.to_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.csv", index=False)

In [44]:
#df_eightgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx")
df_eightgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.csv")

In [45]:
df_eightgram_merge2 = df_eightgram_text[~(df_eightgram_text["ninegram"].isnull() & df_eightgram_text["tengram"].isnull())]
df_eightgram_merge2

,eightgram,frequency,ninegram,tengram
0,bir iki üç dört beş altı yedi sekiz,1060,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,dokuz sekiz yedi altı beş dört üç iki,513,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
2,sekiz yedi altı beş dört üç iki bir,497,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
3,iki üç dört beş altı yedi sekiz dokuz,411,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
4,üç dört beş altı yedi sekiz dokuz on,329,"['iki üç dört beş altı yedi sekiz dokuz on', '...",['bir iki üç dört beş altı yedi sekiz dokuz on...
...,...,...,...,...
109446,bu bok en uzun süre ne kadar etki,7,NaN,['bu bok en uzun süre ne kadar etki gösterdi s...
109448,şu anda telefona gelemez ama beni bıraktığınız...,7,NaN,['joshua şu anda telefona gelemez ama beni bır...
109455,istediğim elini içeri sokman ve kabloyu nazikç...,7,NaN,['senden istediğim elini içeri sokman ve kablo...
109462,insan diğer bir insanın işine çomak sokmak zor...,7,NaN,['neden bir insan diğer bir insanın işine çoma...


In [46]:
#df_eightgram_merge2.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)
df_eightgram_merge2.to_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.csv", index=False)

##### Ninegram

In [47]:
# parameter
source_wordgroup_list = ninegram_list
search_column_name = "ninegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [48]:
search_list = list(searchlist)
result_list = list(resultlist)
df_nine_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_nine_in_ten

,ninegram,tengram
0,ve büyük oğlu öldürülmeden önceki akşamüstü on...,leeds ve büyük oğlu öldürülmeden önceki akşamü...
1,hastayı kurtarmak için ne kadarını kesmek zoru...,böylece hastayı kurtarmak için ne kadarını kes...
2,darby için anlaşma yapma şansın şimdi olsa yin...,darby için anlaşma yapma şansın şimdi olsa yin...
3,muhtemelen bana gönderdikleri beş kutuyu da öd...,şimdi muhtemelen bana gönderdikleri beş kutuyu...
4,uzun zamandır çalışıyorum ve kazandığım parayl...,çünkü uzun zamandır çalışıyorum ve kazandığım ...
...,...,...
20255,ah beni dinle bak belki seninle biraz sohbet e...,ah beni dinle bak belki seninle biraz sohbet e...
20256,eğer başına bir şey gelirse kızımın başına nel...,şimdi eğer başına bir şey gelirse kızımın başı...
20257,içeriye girdiğinde sanki odanın dönüyormuş gib...,o içeriye girdiğinde sanki odanın dönüyormuş g...
20258,aşkın ne olduğunu bu kadar iyi bilseydin neden...,aşkın ne olduğunu bu kadar iyi bilseydin neden...


In [49]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

,ninegram,tengram
0,a onlardan biri olabilir ve b hala kızgın olab...,[çünkü a onlardan biri olabilir ve b hala kızg...
1,a takımı değil belki ama elimizden gelenin en ...,[a takımı değil belki ama elimizden gelenin en...
2,abc üçgeni a kenarı b kenarı ve c kenarına,[abc üçgeni a kenarı b kenarı ve c kenarına eş...
3,abd hava üssünde patlayacak atom bombası daha ...,[abd hava üssünde patlayacak atom bombası daha...
4,abd hükümeti ile alakalı gizli görevler için y...,[abd hükümeti ile alakalı gizli görevler için ...
...,...,...
20237,şükran gününde ve noelde masada sana da bir yer,[şükran gününde ve noelde masada sana da bir y...
20238,şükürler olsun ki bu kiliseyi koruması için bi...,[tanrıya şükürler olsun ki bu kiliseyi korumas...
20239,şükürler olsun ki o zamandan bu yana uzun yollar,[şükürler olsun ki o zamandan bu yana uzun yol...
20240,şüphen varsa beni vur parayı al ve güzelce uyu,[bundan şüphen varsa beni vur parayı al ve güz...


In [50]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [51]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

,ninegram,frequency,tengram
0,dokuz sekiz yedi altı beş dört üç iki bir,463,"[on dokuz sekiz yedi altı beş dört üç iki bir,..."
1,bir iki üç dört beş altı yedi sekiz dokuz,405,"[bir iki üç dört beş altı yedi sekiz dokuz on,..."
2,iki üç dört beş altı yedi sekiz dokuz on,287,[bir iki üç dört beş altı yedi sekiz dokuz on]
3,on dokuz sekiz yedi altı beş dört üç iki,277,[on dokuz sekiz yedi altı beş dört üç iki bir]
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134,NaN
...,...,...,...
40393,artık sana kan nakli yapacak doktor hoffman da...,7,NaN
40394,senin ülken için potansiyel terörist teröristi...,7,NaN
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7,[sabah dokuz gibi geçerken uğrarım bu yüzden ö...
40396,herkes benim iyi adam olduğuma inanmakta bu ka...,7,[neden herkes benim iyi adam olduğuma inanmakt...


In [52]:
#df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx", index=False)
df_ninegram_merge.to_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.csv", index=False)

In [53]:
#df_ninegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx")
df_ninegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.csv")

In [54]:
df_ninegram_merge2 = df_ninegram_text[~(df_ninegram_text["tengram"].isnull())]
df_ninegram_merge2

,ninegram,frequency,tengram
0,dokuz sekiz yedi altı beş dört üç iki bir,463,['on dokuz sekiz yedi altı beş dört üç iki bir...
1,bir iki üç dört beş altı yedi sekiz dokuz,405,['bir iki üç dört beş altı yedi sekiz dokuz on...
2,iki üç dört beş altı yedi sekiz dokuz on,287,['bir iki üç dört beş altı yedi sekiz dokuz on']
3,on dokuz sekiz yedi altı beş dört üç iki,277,['on dokuz sekiz yedi altı beş dört üç iki bir']
6,i t a r i s y e n,128,['g i t a r i s y e n']
...,...,...,...
40382,sevgili kızım aşık bir kadın için bile mantıks...,7,['sevgili kızım aşık bir kadın için bile mantı...
40391,çene çalmayı bir yana bırakıp bu işi gerçekten...,7,['çene çalmayı bir yana bırakıp bu işi gerçekt...
40392,merak etme böyle bir şey yapmamı istemediğinde...,7,['merak etme böyle bir şey yapmamı istemediğin...
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7,['sabah dokuz gibi geçerken uğrarım bu yüzden ...


In [55]:
#df_ninegram_merge2.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)
df_ninegram_merge2.to_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.csv", index=False)

#### Copy Move And Delete

In [57]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Sevengram_Text_Selection_Result.csv',
 'Turkish_Eightgram_Text_Selection_Result.csv',
 'Turkish_Ninegram_Text_Selection_Result.csv',
 'Turkish_Word_Text_Selection_Result.csv']

In [58]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [59]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass